In [1]:
import pandas as pd

df = pd.read_csv('data-p3/dat_P_sub_c.csv')
df.head()


,Pass_ID,valid_P_ID,Airfield,S2,Wait_Time,C_Start,C0,C_avg,Sch_Departure,Act_Departure,...,Day_of_Week,Month,Season,Year,WT_flag,S2_Sch_Flag,S2_Act_Flag,Sch_Act_Flag,Flight_ID,Delay_in_Seconds
0,5348206,1,AUC,2028-09-01 6:37,NaN,1,1,1.0,2028-09-01 8:06,2028-09-01 8:06,...,5 - FRI,09 - SEP,3 - SUMMER,2028,1,0,0,0,18095,0
1,5348209,1,AUC,2028-09-01 6:38,NaN,1,1,1.0,2028-09-01 8:06,2028-09-01 8:06,...,5 - FRI,09 - SEP,3 - SUMMER,2028,1,0,0,0,18095,0
2,5348216,1,AUC,2028-09-01 6:39,NaN,1,1,1.0,2028-09-01 8:06,2028-09-01 8:06,...,5 - FRI,09 - SEP,3 - SUMMER,2028,1,0,0,0,18095,0
3,5348224,1,AUC,2028-09-01 6:39,NaN,1,1,1.0,2028-09-01 8:06,2028-09-01 8:06,...,5 - FRI,09 - SEP,3 - SUMMER,2028,1,0,0,0,18095,0
4,5348225,1,AUC,2028-09-01 6:40,NaN,1,1,1.0,2028-09-01 8:06,2028-09-01 8:06,...,5 - FRI,09 - SEP,3 - SUMMER,2028,1,0,0,0,18095,0


In [2]:
df['S2'] = pd.to_datetime(df['S2'])
df['Wait_Time'] = pd.to_timedelta(df['Wait_Time'], unit='m')
df['S1'] =df['S2'] - df['Wait_Time']

In [3]:
# df = df.sort_values(by='S1')s

In [4]:
def determine_cluster(timestamp):

    is_weekend = timestamp.weekday() >= 5  
    hour = timestamp.hour
    
    if 0 <= hour < 4:
        cluster = "0:00 - 4:00"
    elif 4 <= hour < 8:
        cluster = "4:00 - 8:00"
    elif 8 <= hour < 12:
        cluster = "8:00 - 12:00"
    elif 12 <= hour < 16:
        cluster = "12:00 - 16:00"
    elif 16 <= hour < 20:
        cluster = "16:00 - 20:00"
    else:
        cluster = "20:00 - 0:00"
    
    return f"{'Weekend' if is_weekend else 'Weekday'} - {cluster}"

df['cluster'] = df['S2'].apply(determine_cluster)

In [5]:
df['cluster'].isna().any()

False

In [6]:
def cluster_hours(timestamp):
    is_weekend = timestamp.weekday() >= 5
    
    return 104 if is_weekend else 260

df['cluster_hours'] = df['S2'].apply(cluster_hours)

In [7]:
df[['cluster','cluster_hours']]

,cluster,cluster_hours
0,Weekday - 4:00 - 8:00,260
1,Weekday - 4:00 - 8:00,260
2,Weekday - 4:00 - 8:00,260
3,Weekday - 4:00 - 8:00,260
4,Weekday - 4:00 - 8:00,260
...,...,...
114127,Weekend - 16:00 - 20:00,104
114128,Weekend - 16:00 - 20:00,104
114129,Weekend - 16:00 - 20:00,104
114130,Weekend - 16:00 - 20:00,104


In [8]:
arrival_counts = df.groupby('cluster').size()
arrival_counts.head()


cluster
Weekday - 12:00 - 16:00    21850
Weekday - 16:00 - 20:00    20987
Weekday - 20:00 - 0:00      1544
Weekday - 4:00 - 8:00      15203
Weekday - 8:00 - 12:00     20784
dtype: int64

In [9]:
results = arrival_counts.to_frame(name='count')

results = results.merge(df[['cluster', 'cluster_hours']].drop_duplicates(), on='cluster', how='left')
results.head()


,cluster,count,cluster_hours
0,Weekday - 12:00 - 16:00,21850,260
1,Weekday - 16:00 - 20:00,20987,260
2,Weekday - 20:00 - 0:00,1544,260
3,Weekday - 4:00 - 8:00,15203,260
4,Weekday - 8:00 - 12:00,20784,260


In [10]:
results['avg_arrival_rate'] = results['count'] / results['cluster_hours']
results


,cluster,count,cluster_hours,avg_arrival_rate
0,Weekday - 12:00 - 16:00,21850,260,84.038462
1,Weekday - 16:00 - 20:00,20987,260,80.719231
2,Weekday - 20:00 - 0:00,1544,260,5.938462
3,Weekday - 4:00 - 8:00,15203,260,58.473077
4,Weekday - 8:00 - 12:00,20784,260,79.938462
5,Weekend - 12:00 - 16:00,9492,104,91.269231
6,Weekend - 16:00 - 20:00,10202,104,98.096154
7,Weekend - 20:00 - 0:00,626,104,6.019231
8,Weekend - 4:00 - 8:00,5972,104,57.423077
9,Weekend - 8:00 - 12:00,7472,104,71.846154


In [11]:
average_C0 = df.groupby('cluster')['C0'].mean().reset_index(name='avg_C0')
average_C0

,cluster,avg_C0
0,Weekday - 12:00 - 16:00,1.235103
1,Weekday - 16:00 - 20:00,1.372278
2,Weekday - 20:00 - 0:00,1.022668
3,Weekday - 4:00 - 8:00,1.830626
4,Weekday - 8:00 - 12:00,1.264049
5,Weekend - 12:00 - 16:00,1.203118
6,Weekend - 16:00 - 20:00,1.465595
7,Weekend - 20:00 - 0:00,1.000000
8,Weekend - 4:00 - 8:00,1.776959
9,Weekend - 8:00 - 12:00,1.154042


In [12]:
distribution_C0 = pd.crosstab(df['cluster'], df['C0'], normalize='index') * 100
distribution_C0.columns = [f'{col}_total_servers' for col in distribution_C0.columns]
distribution_C0


,1_total_servers,2_total_servers,3_total_servers
cluster,,,
Weekday - 12:00 - 16:00,77.066362,22.356979,0.576659
Weekday - 16:00 - 20:00,63.930052,34.912088,1.157860
Weekday - 20:00 - 0:00,97.733161,2.266839,0.000000
Weekday - 4:00 - 8:00,20.384135,76.169177,3.446688
Weekday - 8:00 - 12:00,74.576597,24.441878,0.981524
Weekend - 12:00 - 16:00,79.688158,20.311842,0.000000
Weekend - 16:00 - 20:00,57.371104,38.698294,3.930602
Weekend - 20:00 - 0:00,100.000000,0.000000,0.000000
Weekend - 4:00 - 8:00,24.966510,72.371065,2.662425


In [13]:
results = average_C0.merge(distribution_C0, on='cluster', how='left')
results

,cluster,avg_C0,1_total_servers,2_total_servers,3_total_servers
0,Weekday - 12:00 - 16:00,1.235103,77.066362,22.356979,0.576659
1,Weekday - 16:00 - 20:00,1.372278,63.930052,34.912088,1.157860
2,Weekday - 20:00 - 0:00,1.022668,97.733161,2.266839,0.000000
3,Weekday - 4:00 - 8:00,1.830626,20.384135,76.169177,3.446688
4,Weekday - 8:00 - 12:00,1.264049,74.576597,24.441878,0.981524
5,Weekend - 12:00 - 16:00,1.203118,79.688158,20.311842,0.000000
6,Weekend - 16:00 - 20:00,1.465595,57.371104,38.698294,3.930602
7,Weekend - 20:00 - 0:00,1.000000,100.000000,0.000000,0.000000
8,Weekend - 4:00 - 8:00,1.776959,24.966510,72.371065,2.662425
9,Weekend - 8:00 - 12:00,1.154042,84.595824,15.404176,0.000000


In [15]:
def imputation_cluster(timestamp):

    day_name = timestamp.strftime('%A')  # day of week
    hour = timestamp.hour
    
    if 0 <= hour < 4:
        time_cluster = "0:00 - 4:00"
    elif 4 <= hour < 8:
        time_cluster = "4:00 - 8:00"
    elif 8 <= hour < 12:
        time_cluster = "8:00 - 12:00"
    elif 12 <= hour < 16:
        time_cluster = "12:00 - 16:00"
    elif 16 <= hour < 20:
        time_cluster = "16:00 - 20:00"
    else:
        time_cluster = "20:00 - 0:00"
    

    return f"{day_name} - {time_cluster}"

df['imputation_cluster'] = df['S2'].apply(imputation_cluster)


In [16]:
imp_avg_wait = df.groupby('imputation_cluster')['Wait_Time'].mean()

df['wait_time_imp'] = df['Wait_Time'].fillna(df['imputation_cluster'].map(imp_avg_wait))
df['wait_time_imp'].hasnans


False

In [17]:
df['wait_time_imp'] = df['wait_time_imp'] / pd.Timedelta(minutes=1)

thresholds = [5, 10, 15, 20, 25, 30]

results = {}

for cluster, group in df.groupby('cluster'):
    results[cluster] = {f"<= {threshold} mins": (group['wait_time_imp'] <= threshold).mean() * 100
                        for threshold in thresholds}

percentage_table_by_cluster = pd.DataFrame(results).transpose()
percentage_table_by_cluster.index.name = 'cluster'
percentage_table_by_cluster.reset_index(inplace=True)
percentage_table_by_cluster


,cluster,<= 5 mins,<= 10 mins,<= 15 mins,<= 20 mins,<= 25 mins,<= 30 mins
0,Weekday - 12:00 - 16:00,72.311213,92.713959,97.748284,99.322654,99.615561,99.835240
1,Weekday - 16:00 - 20:00,78.791633,96.088055,99.047029,99.733168,99.918997,99.966646
2,Weekday - 20:00 - 0:00,96.891192,100.000000,100.000000,100.000000,100.000000,100.000000
3,Weekday - 4:00 - 8:00,40.373610,74.347168,90.153259,96.704598,99.046241,99.671118
4,Weekday - 8:00 - 12:00,58.766359,85.484026,93.523865,97.329677,98.720169,99.629523
5,Weekend - 12:00 - 16:00,74.262537,92.899284,98.714707,99.873578,99.989465,99.989465
6,Weekend - 16:00 - 20:00,63.712997,94.961772,97.941580,98.853166,99.480494,99.872574
7,Weekend - 20:00 - 0:00,98.242812,99.840256,100.000000,100.000000,100.000000,100.000000
8,Weekend - 4:00 - 8:00,49.866042,88.580040,97.906899,99.648359,99.949766,99.949766
9,Weekend - 8:00 - 12:00,67.277837,92.036938,97.430407,99.156852,99.692184,99.973233
